In [1]:
import torch, os, tqdm
import numpy as np
import pandas as pd
import torch.nn.functional as F


from PIL import Image
from data import TRdataset, dataset
from utils import ocr_transformer, val_transformer, encoder
from torch.utils.data import DataLoader, Dataset

from models import VitOCR #, SeArchitecture
from utils import save_model


device = "cuda" if torch.cuda.is_available() else "cpu"

C:\Users\yerda\anaconda3\lib\site-packages\albumentations\imgaug\transforms.py:231: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
C:\Users\yerda\anaconda3\lib\site-packages\albumentations\imgaug\transforms.py:174: FutureWarning: This augmentation is deprecated. Please use Emboss instead
  warnings.warn("This augmentation is deprecated. Please use Emboss instead", FutureWarning)


In [2]:
val = pd.read_csv("dataset/val_annotation.csv")
test = pd.read_csv("dataset/test_annotation.csv")
train = pd.read_csv("dataset/train_annotation.csv")
train.drop(8, inplace=True)
train.drop(64, inplace=True)
train.drop(117, inplace=True)
train.drop(213, inplace=True)

In [3]:
trainLoader = DataLoader(TRdataset(train, encoder, max_sequence=19, transformer=val_transformer), batch_size=64, shuffle=True)
valLoader = DataLoader(TRdataset(val, encoder, max_sequence=19, transformer=val_transformer), batch_size=64, shuffle=False)
testLoader = DataLoader(TRdataset(test, encoder, max_sequence=19, transformer=val_transformer), batch_size=64, shuffle=False)

In [4]:
model = VitOCR().to(device)
# model = SeArchitecture(in_channels=9).to(device)
loss_fn = torch.nn.CTCLoss(blank=0, reduction="mean", zero_infinity=False)
optimizer = torch.optim.Adam(model.parameters(), 1e-3)

In [3]:
os.makedirs("models", exist_ok=True)

In [6]:
num_epochs, prev_loss = 10000, torch.inf
train_loss, val_loss = [], []

for epoch in tqdm.trange(num_epochs):
    model.train()
    loss_list, count = 0, 0
    for images, targets, input_lengths, target_lengths in trainLoader:
        images = images.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        predictions = model(images)
        predictions = F.log_softmax(predictions, dim=2)
        loss = loss_fn(predictions.permute(1, 0, 2), targets, input_lengths, target_lengths)
        loss.backward()
        optimizer.step()
        loss_list += loss.cpu().item()
        count += 1
    loss = loss_list/count
    train_loss.append(loss)
    
    model.eval()
    @torch.no_grad()
    def validation():
        val_loss, val_counter = 0, 0
        for images, targets, input_lengths, target_lengths in valLoader:
            images = images.to(device)
            targets = targets.to(device)
            predictions = model(images)
            predictions = F.log_softmax(predictions, dim=2)
            loss = loss_fn(predictions.permute(1, 0, 2), targets, input_lengths, target_lengths)
            val_loss += loss.cpu().item()
            val_counter += 1
        return val_loss / val_counter
    v_loss = validation()
    val_loss.append(v_loss)
    prev_loss = save_model("attention_model", epoch, prev_loss, v_loss, model, optimizer)
    
    print(f"Epoch: {epoch} | Training loss {loss} | Validation loss {v_loss}")

  0%|                                                                             | 1/10000 [00:24<67:34:48, 24.33s/it]

The best model was saved!
Epoch: 0 | Training loss 3.722171388566494 | Validation loss 3.0559626817703247



  0%|                                                                             | 2/10000 [00:32<41:35:33, 14.98s/it]

The best model was saved!
Epoch: 1 | Training loss 3.4047480821609497 | Validation loss 3.0326748192310333



  0%|                                                                             | 3/10000 [00:41<33:39:37, 12.12s/it]

...

Epoch: 4966 | Training loss 2.862708255648613 | Validation loss 2.7916559875011444



 50%|███████████████████████████████████▎                                   | 4968/10000 [11:15:59<11:35:37,  8.29s/it]

Epoch: 4967 | Training loss 2.8702483773231506 | Validation loss 2.7911561727523804


 50%|███████████████████████████████████▎                                   | 4968/10000 [11:16:04<11:24:46,  8.17s/it]


KeyboardInterrupt: 

In [ ]:
from torchsummary import summary

In [ ]:
summary(model, (9, 50, 224), batch_size=64)

In [7]:
# pd.DataFrame({
#     "epoch": range(0, len(val_loss)),
#     "train_loss": train_loss[:],
#     "val_loss": val_loss,
# }).to_csv("Attentionlosses.csv", index=False)